In [1365]:
import cv2 as cv
import numpy as np
import pytesseract
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageFilter
import os

%matplotlib qt

In [1366]:
def img_imread(img_path):
    car_plate_img = cv.imread(img_path)
    car_plate_img = cv.cvtColor(car_plate_img, cv.COLOR_BGR2RGB)

    return car_plate_img

In [1367]:
def plate_to_string(plate):
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

    config = f'--psm 6 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    return pytesseract.image_to_string(plate, config=config)

In [1368]:
def enlarge_img(image, scale_percent):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)

    dim = (width, height)
    resized_image = cv.resize(image, dim, interpolation=cv.INTER_AREA)

    return resized_image

In [1369]:
def car_plate_extract(image, cascade, num=1):
        car_plate_rects = cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5)
        x0 = 0; y0 = 0; w0 = 0; h0 = 0
        car_plate_img = []

        # find all variants with plates on image
        for coord in car_plate_rects:
            x, y, w, h = coord

            # the largest area of car plate
            if w0 * h0 < w * h:
                x0 = x; y0 = y; w0 = w; h0 = h
                car_plate_img = image[y0:y0+h0, x0:x0+w0]

        cv.rectangle(image, (x0, y0), (x0+w0, y0+h0), (0, 255, 0), 2)

        plt.figure(num)
        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.axis('off')

        plt.subplot(1, 2, 2)
        img_plate = enlarge_img(car_plate_img, 1250)
        rgb_plate = cv.cvtColor(img_plate, cv.COLOR_BGR2RGB)
        plate_gray = cv.cvtColor(rgb_plate, cv.COLOR_RGB2GRAY)
        plate_blur = cv.medianBlur(plate_gray, 3)
        img_binary = cv.threshold(plate_blur, 158, 255, cv.THRESH_BINARY)[1]

        plt.imshow(img_binary)
        plt.axis('off')
        plt.show()

        return plate_to_string(img_plate)

In [1370]:
def cascade_haar(path):
    cascade = cv.CascadeClassifier(path)
    return cascade

In [1371]:
def show_gallery(path_images, haar_path):
    plates = []
    for i, file in enumerate(os.listdir(path_images)):
        if file.endswith('.jpg'):
            image = img_imread(path_images+file)
            plates.append(car_plate_extract(image, cascade_haar(haar_path), num=i+1))

    return plates

In [1372]:
print(f"Car plates: {show_gallery('images/cars_plates/', 'data/haarcascade_russian_plate_number.xml')}")

Car plates: ['H023BM194\n', '', 'TSTYCS3LE\n', '', 'C666CC4\n']
